### Dependencies and Imports

In [1]:
#Dependencies and Imports
import spotipy
import requests
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import pprint as pprint
import pandas as pd
import json
import os
import csv
import sys

### ID and Password for accessing Spotify API

In [2]:
### ID and Password for accessing Spotify API
client_id = "7f7553c4d1b64d36952992adbf5cb2d0"
client_secret = "ac14b01e359d4446b51bfb69aaab5d81"

# Replace Spotify access token each hour!
access_token = "BQAoaapRZq5Gv0IshSLNh_6iTAHG64jdul6NQWBLs71MAt3c3XZKddNxjhCYUuHZ5OdtSxOjpql2dUq2uK6V7zPk-7JoKUwfQJrDf-KsK6A8-NIhGqZtsKzD4gND9AQn9CSG4vnwG8y9LiExEyE8zK6wT8pe_nt_W3Nyt53PA7rTg73cEFv7WwkYY3kl-UBxUTkoOGQr9EVdJ4SSAJ67x48QfdEqYUIvAXveaTODKI31mRK-WuGXCJzgVjHf1jH1O3oBcNPK"

# Setup the credentials
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

# Make the connection
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


### Lists to store data

In [3]:
# Lists to store data
sp_id=[]     
followers_count=[]
genres=[]
popularity=[]
sp_name=[]

### Create Dataframe from Sentiment Analysis csv

In [4]:
# Load Billboard csv and Create Dataframe
csv_path = "billboard_sentanalys.csv"

df=pd.read_csv(csv_path,encoding="ANSI")

print(df.head())

                                      Title                         Artist  \
0                               wooly bully  sam the sham and the pharaohs   
1  i cant help myself sugar pie honey bunch                      four tops   
2                i cant get no satisfaction             the rolling stones   
3                       you were on my mind                        we five   
4              youve lost that lovin feelin         the righteous brothers   

   Year  Rank                                             Lyrics  Source  \
0  1965     1  sam the sham miscellaneous wooly bully wooly b...     3.0   
1  1965     2  Ooh, sugar pie, honey bunch You know that I lo...     1.0   
2  1965     3  I can't get no satisfaction, I can't get no sa...     1.0   
3  1965     4  Verse 1 When I woke up this morning You were o...     1.0   
4  1965     5  You never close your eyes anymore when I kiss ...     1.0   

                                            CheckURL  comp_score  pos_scor

In [5]:
#Get Artist from CheckURL (since we want to analyze based on main artist; not featured ppl)
mainartist = df["CheckURL"].replace(to_replace=["http://www.metrolyrics.com/","-",".html"],value=[""," ",""],regex=True)

mainartist = [item.split() for item in mainartist]
 
    
#print(mainartist)
for i,row in enumerate(mainartist):
    
    for index,item in enumerate(row):
        
        if item == "lyrics":
            mainartist[i] = row[(index+1):]
            
            break

mainartist = [" ".join(row) for row in mainartist]

mainartist_df = pd.DataFrame({"main_artist":mainartist})

print(mainartist_df.head())

              main_artist
0            sam the sham
1               four tops
2      the rolling stones
3                 we five
4  the righteous brothers


In [6]:
#add main artist list to current dataframe

df = pd.concat([df,mainartist_df], axis=1)

df.head()

,Title,Artist,Year,Rank,Lyrics,Source,CheckURL,comp_score,pos_score,neu_score,neg_score,main_artist
0,wooly bully,sam the sham and the pharaohs,1965,1,sam the sham miscellaneous wooly bully wooly b...,3.0,http://www.metrolyrics.com/wooly-bully-lyrics-...,-0.9946,0.020,0.628,0.352,sam the sham
1,i cant help myself sugar pie honey bunch,four tops,1965,2,"Ooh, sugar pie, honey bunch You know that I lo...",1.0,http://www.metrolyrics.com/i-cant-help-myself-...,0.9586,0.149,0.749,0.102,four tops
2,i cant get no satisfaction,the rolling stones,1965,3,"I can't get no satisfaction, I can't get no sa...",1.0,http://www.metrolyrics.com/i-cant-get-no-satis...,0.9142,0.216,0.664,0.120,the rolling stones
3,you were on my mind,we five,1965,4,Verse 1 When I woke up this morning You were o...,1.0,http://www.metrolyrics.com/you-were-on-my-mind...,-0.9607,0.057,0.769,0.174,we five
4,youve lost that lovin feelin,the righteous brothers,1965,5,You never close your eyes anymore when I kiss ...,1.0,http://www.metrolyrics.com/youve-lost-that-lov...,0.9840,0.207,0.726,0.066,the righteous brothers


In [18]:
#Groupby artist name
artist_group = df.groupby(["main_artist"]).mean()

artist_group = artist_group.reset_index()

artist_group = artist_group.assign(followers="",popularity="",sp_name="",sp_id="",genre="")

artist_group.head()

,main_artist,Year,Rank,Source,comp_score,pos_score,neu_score,neg_score,followers,genre,popularity,sp_id,sp_name
0,100 proof aged in soul,1970.000000,84.000000,NaN,NaN,NaN,NaN,NaN,,,,,
1,10000 maniacs,1994.000000,40.000000,1.000000,0.993600,0.234000,0.699000,0.066000,,,,,
2,10cc,1976.000000,43.000000,1.000000,0.995500,0.191000,0.771000,0.038000,,,,,
3,112,1999.833333,50.500000,1.333333,0.668350,0.146500,0.796000,0.057500,,,,,
4,1910 fruitgum company,1968.333333,49.333333,1.666667,0.496667,0.189333,0.731333,0.079333,,,,,


In [21]:
artist_group.set_value(4,artist_group["followers"],"merp")

artist_group.head()

KeyError: "['' '' '' ..., '' '' ''] not in index"

### Initial Spotify search

### Append Spotify Results to Lists followers_count. genres, popularity

In [9]:
# Append Spotify Results to Lists
for index,i in enumerate(artist_group["main_artist"]):
    result = sp.search(i,limit=1, offset=0, type='artist', market="US")
    #print(result)
    sub=result['artists']["items"]
    
    df=df.set_value(index,df["followers"],sub[0]["followers"]["total"])

    popularity.append(sub[0]["popularity"])
    sp_name.append(sub[0]["name"])
    genres.append(sub[0]["genres"])
    sp_id.append(sub[0]["id"])


print(genres)

IndexError: list index out of range

### Zip up our lists

In [ ]:
#cleaned_csv = zip(id,title,song_count,followers_count,genres,popularity)

cleaned_csv = zip(artist_group["main_artist"],artist_group["Year"],artist_group["Rank"],
                  artist_group["comp_score"], artist_group["pos_score"],artist_group["neu_score"],
                  artist_group["neg_score"], sp_id, sp_name, followers_count,popularity, genres)
print(cleaned_csv)

### Send Data to output_file

In [ ]:
# Send Data to output file
output_file = os.path.join("billboard_artistfinal.csv")
with open(output_file, "w", newline="", encoding="UTF-8") as datafile:
    writer = csv.writer(datafile)
    
    writer.writerow(["main_artist", "avg_year", "avg_rank","avg_comp_score","avg_pos_score","avg_neu_score",
                     "avg_neg_score","sp_id","sp_name","followers_count", "popularity","genres"])
    
    writer.writerows(cleaned_csv)